In [1]:
## run this if you get an error for missing imports
! pip install gradio
! pip install numpy
! pip install pickle
! pip install torch
! pip install spacy
! python -m spacy download en

c:\Users\Josep\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import pickle
import torch
from LSTM_Architecture import NLP_LSTM
import spacy

c:\Users\Josep\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nlp = spacy.load("en_core_web_sm")
file = open('tok_to_work', 'rb')
tok_convert = pickle.load(file)
file.close()

In [3]:
file = open('tok_track', 'rb')
tok_track = pickle.load(file)
file.close()

In [4]:
gpu = torch.cuda.is_available()
model = NLP_LSTM(gpu, 150, 150, len(tok_convert), 4, 0.44, True)
info = torch.load('latest_model.pt', map_location=torch.device('cpu'))
model.load_state_dict(info['model'])


<All keys matched successfully>

In [5]:
class data_processor:
    def __init__(self, tokenized_df, conv, track):
        self.word_idx = conv
        self.tracker = track
        self.most = 29
        self.threshold = 4
        normalized = tokenized_df['tweet'].apply(data_processor.sentence_normalizer, args=(self,))
        normalized_df = tokenized_df.assign(tweet = normalized)
        numerized = normalized_df['tweet'].apply(data_processor.numerizer, args=(self,))
        self.text = normalized_df.assign(numerized_tweet = numerized)


    def sentence_normalizer(sentence, self):
        final_tok = []
        count = 0 
        for token in sentence:
            final_tok.append(token)
            count+=1
            if count >= self.most:
                break
        if len(final_tok)<self.most:
            final_tok.extend(['<PAD>']*(self.most-len(final_tok)))
        return final_tok


    def numerizer(x, self):
        base = []
        for token in x:
            try:
                if token.norm in self.tracker:
                    if self.tracker[token.norm]>= self.threshold:
                        base.append(self.word_idx[str(token)])
                    else:
                        base.append(self.word_idx['<UNK>'])
                else:
                    base.append(self.word_idx['<UNK>'])
            except:
                base.append(self.word_idx['<PAD>'])

        return base

In [6]:
def run(Text):
        repl = {'@\w*': ' ', '&amp;' : 'and','\su\s':' you ', '&#\w*;': ' ', 
        '#':' ', '\s2\s': 'two', 'bihday':"birthday", "ð[^ ]*": ' ' ,
        "â[^ ]*": ' ',"(dont)|(don't)": 'do not', "(cant)|(can't)": "can not",
        "(yous)|(you's)": "you is", "(yous)|(you's)": "you is", 
        "(youve)|(you've)": "you have", "(doesnt)|(doesn't)": 'does not', 
        "(wont)|(won't)": 'will not', "[0-9]+\.*[0-9%]+\w*" : "NUMBER",'\\n\.':' ' ,'\\n':' ',
        "\.{2,}": '.', "!{2,}":'!', "\?{2,}":'?', 'ing[^a-z]':' ', 'ed[^a-z]': ' ', '_':" ",
        ' +': ' '}

        text = pd.DataFrame({'tweet': [Text]})
        cleaned = text['tweet'].str.lower()
        cleaned = cleaned.replace(repl, regex=True)
        text = text.assign(tweet = cleaned)
        cleaned = text['tweet'].apply(lambda x: nlp(x.strip()))
        text = text.assign(tweet = cleaned)
        tok_convert
        text = data_processor(text, tok_convert, tok_track)
        text = text.text
        text = torch.tensor(np.stack(text['numerized_tweet']))
        text = torch.round(model.tester(text)).item()
        if text == 1:
                return 'This is Toxic'
        else:
                return 'This is not Toxic'
        



In [7]:
outputs = gr.outputs.Textbox()
inputs = gr.inputs.Textbox()
app = gr.Interface(fn=run, inputs=[inputs], outputs=outputs,description="This is a Sentiment Analysis Model")

c:\Users\Josep\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
c:\Users\Josep\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\Josep\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Users\Josep\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [8]:
app.launch(share = True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app
